In [ ]:
# Cài đặt Unsloth nếu chưa có
# !pip install unsloth
# !pip install torch torchvision torchaudio
# !pip install transformers datasets pillow


In [ ]:
# Import các thư viện cần thiết
from unsloth import FastVisionModel  # FastVisionModel for vision-language models
import torch
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import Dataset
from PIL import Image
import numpy as np
import json
import os
import gc

print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


In [ ]:
# Load model và processor theo requirements
model, processor = FastVisionModel.from_pretrained(
    "unsloth/gemma-3n-E4B",
    load_in_4bit=True,  # Use 4bit để giảm memory usage. False for 16bit LoRA.
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for long context
    max_seq_length=2048,  # Có thể điều chỉnh tùy theo nhu cầu
    dtype=None,  # Auto detect
    # token="hf_...",  # Sử dụng nếu model gated
)

print("✅ Model và processor đã được load thành công!")
print(f"Model: {model.__class__.__name__}")
print(f"Processor: {processor.__class__.__name__}")


In [ ]:
# Configure model cho fine-tuning theo requirements
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,     # False nếu không fine-tuning vision layers
    finetune_language_layers=True,   # False nếu không fine-tuning language layers
    finetune_attention_modules=True, # False nếu không fine-tuning attention layers
    finetune_mlp_modules=True,       # False nếu không fine-tuning MLP layers

    r=32,                           # The larger, the higher the accuracy, but might overfit
    lora_alpha=32,                  # Recommended alpha == r at least
    lora_dropout=0,
    bias="none",
    random_state=3407,
    use_rslora=False,               # We support rank stabilized LoRA
    loftq_config=None,              # And LoftQ
    target_modules="all-linear",    # Optional now! Can specify a list if needed
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)

print("✅ PEFT model configuration hoàn thành!")

# Print trainable parameters
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"Trainable params: {trainable_params:,} || All params: {all_param:,} || Trainable%: {100 * trainable_params / all_param:.2f}")


In [ ]:
# Tạo dataset mẫu cho vision-language training
def create_sample_images(size=(224, 224), num_images=3):
    """Tạo images mẫu cho demo"""
    images = []
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Red, Green, Blue
    
    for i in range(num_images):
        # Tạo image màu đơn giản
        img_array = np.full((*size, 3), colors[i % len(colors)], dtype=np.uint8)
        img = Image.fromarray(img_array)
        images.append(img)
    
    return images

# Tạo sample dataset
def create_vision_language_dataset():
    """Tạo dataset mẫu cho vision-language fine-tuning"""
    
    # Sample data với format phù hợp cho vision-language model
    sample_data = [
        {
            "messages": [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful vision-language assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Describe what you see in these images:"},
                        {"type": "image", "image": create_sample_images()[0]},
                        {"type": "image", "image": create_sample_images()[1]},
                        {"type": "text", "text": "What are the main colors?"}
                    ]
                },
                {
                    "role": "assistant", 
                    "content": [
                        {"type": "text", "text": "I can see two images with distinct colors. The first image is predominantly red, and the second image is predominantly green. These are solid color images that demonstrate different hues."}
                    ]
                }
            ]
        },
        {
            "messages": [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful vision-language assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Compare these images and tell me the differences:"},
                        {"type": "image", "image": create_sample_images()[0]},
                        {"type": "image", "image": create_sample_images()[2]}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": "The main difference between these two images is their color. The first image is red while the second image is blue. Both images appear to be solid color backgrounds with the same dimensions."}
                    ]
                }
            ]
        },
        {
            "messages": [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful vision-language assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "What do you think about this sequence of images?"},
                        {"type": "image", "image": create_sample_images()[0]},
                        {"type": "image", "image": create_sample_images()[1]},
                        {"type": "image", "image": create_sample_images()[2]}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": "This sequence shows three images with primary colors: red, green, and blue (RGB). This represents the fundamental color components used in digital displays and image processing. The sequence demonstrates the three primary colors of light."}
                    ]
                }
            ]
        }
    ]
    
    return Dataset.from_list(sample_data)

# Tạo dataset
dataset = create_vision_language_dataset()
print(f"✅ Dataset created with {len(dataset)} samples")
print(f"Sample keys: {dataset[0].keys()}")
print(f"First sample structure: {type(dataset[0]['messages'])}")


In [ ]:
def apply_chat_template(examples):
    """Apply chat template cho dataset"""
    texts = []
    for messages in examples["messages"]:
        # Process messages để tạo text format phù hợp
        formatted_text = ""
        
        for message in messages:
            role = message["role"]
            content = message["content"]
            
            if role == "system":
                formatted_text += f"<|system|>\\n{content[0]['text']}\\n"
            elif role == "user":
                formatted_text += f"<|user|>\\n"
                for item in content:
                    if item["type"] == "text":
                        formatted_text += item["text"] + " "
                    elif item["type"] == "image":
                        formatted_text += "<image> "  # Placeholder cho image
                formatted_text += "\\n"
            elif role == "assistant":
                formatted_text += f"<|assistant|>\\n"
                for item in content:
                    if item["type"] == "text":
                        formatted_text += item["text"]
                formatted_text += "\\n"
        
        texts.append(formatted_text)
    
    return {"text": texts}

# Apply formatting
formatted_dataset = dataset.map(
    apply_chat_template,
    batched=True,
    remove_columns=dataset.column_names
)

print("✅ Dataset formatting completed")
print("Sample formatted text:")
print(formatted_dataset[0]["text"][:500] + "...")


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./gemma3n_finetuned",
    per_device_train_batch_size=1,  # Giảm batch size vì vision model tốn memory
    gradient_accumulation_steps=4,   # Để có effective batch size = 4
    warmup_steps=10,
    max_steps=100,  # Số steps ít cho demo, tăng lên cho training thực tế
    learning_rate=2e-5,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    optim="adamw_8bit",  # Optimize memory usage
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    dataloader_pin_memory=False,  # Giảm memory usage
    remove_unused_columns=False,  # Giữ lại columns cho vision data
    seed=3407,
)

print("✅ Training arguments configured")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total training steps: {training_args.max_steps}")
print(f"Learning rate: {training_args.learning_rate}")


In [ ]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=processor.tokenizer,  # Sử dụng tokenizer từ processor
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=1,  # Tránh multiprocessing issues
    packing=False,  # Tắt packing cho vision model
    args=training_args,
)

print("✅ SFTTrainer initialized successfully")
print(f"Model device: {next(model.parameters()).device}")
print(f"Training dataset size: {len(formatted_dataset)}")


In [ ]:
# Bắt đầu training
print("🚀 Starting training...")
print("This may take some time depending on your hardware...")

try:
    # Clear cache trước khi training
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Train model
    trainer_stats = trainer.train()
    
    print("✅ Training completed successfully!")
    print(f"Training loss: {trainer_stats.training_loss:.4f}")
    print(f"Total time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
    
except Exception as e:
    print(f"❌ Training failed with error: {e}")
    print("This might be due to memory constraints or compatibility issues.")
    print("Try reducing batch_size or max_steps.")


In [ ]:
# Save model and processor
output_dir = "./gemma3n_finetuned_final"
os.makedirs(output_dir, exist_ok=True)

try:
    # Save using Unsloth's optimized save method
    model.save_pretrained(output_dir)
    processor.save_pretrained(output_dir)
    
    print(f"✅ Model saved to {output_dir}")
    
    # Optional: Save as merged model for deployment
    print("Saving merged model for deployment...")
    model.save_pretrained_merged(
        f"{output_dir}_merged", 
        processor.tokenizer, 
        save_method="merged_16bit"
    )
    print(f"✅ Merged model saved to {output_dir}_merged")
    
except Exception as e:
    print(f"⚠️ Save error: {e}")
    print("Trying alternative save method...")
    model.save_pretrained(output_dir)
    processor.tokenizer.save_pretrained(output_dir)


In [ ]:
# Test inference với model đã fine-tune
print("🧪 Testing inference...")

try:
    # Enable inference mode
    FastVisionModel.for_inference(model)
    
    # Prepare test input
    test_prompt = """<|system|>
You are a helpful vision-language assistant.
<|user|>
What do you see in this image? <image>
<|assistant|>
"""
    
    # Tokenize input
    inputs = processor.tokenizer(
        test_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    
    # Decode response
    response = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("✅ Inference successful!")
    print("Generated response:")
    print("-" * 50)
    print(response)
    print("-" * 50)
    
except Exception as e:
    print(f"⚠️ Inference error: {e}")
    print("This is expected if using sample data without real images.")


In [ ]:
# Clean up memory
print("🧹 Cleaning up memory...")

del model
del processor
del trainer
del dataset
del formatted_dataset

gc.collect()

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory freed. Available: {torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated():.2f} MB")

print("✅ Cleanup completed")
